In [0]:
%sql
USE demo;


In [0]:
SHOW TABLES IN demo;

In [0]:
CREATE TABLE IF NOT EXISTS drivers 
USING JSON
OPTIONS (path 'dbfs:/FileStore/drivers.json', header 'true');

SELECT * FROM drivers

In [0]:
CREATE TABLE IF NOT EXISTS constructors
USING JSON
OPTIONS (path 'dbfs:/FileStore/constructors.json', header 'true');

SELECT * FROM constructors

In [0]:
%python
if not spark.catalog.tableExists("races"):
    df = spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/FileStore/races.csv")
    df.write.saveAsTable("races")

In [0]:
SELECT * FROM races;

In [0]:
CREATE TABLE  IF NOT EXISTS results
USING JSON
OPTIONS (path 'dbfs:/FileStore/results.json', header 'true');

In [0]:
SELECT * FROM results;

In [0]:
SHOW TABLES IN demo;

In [0]:
CREATE TABLE IF NOT EXISTS calcuated_race_results
USING parquet
AS
SELECT races._c1 AS race_year,
       constructors.name AS team_name, drivers.name AS driver_name, results.position, results.points,
       11-results.points AS calcuated_points
FROM races
JOIN results ON races._c0 = results.raceId
JOIN constructors ON results.constructorId = constructors.constructorId
JOIN drivers ON results.driverId = drivers.driverId
WHERE results.position<=10